In [1]:
import cv2 as cv
import numpy as np
import glob
import os
import imutils
from imutils.perspective import four_point_transform
from skimage.segmentation import clear_border

In [2]:
def print_img(i,img):
    cv.imshow(str(i),img)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [3]:
def find_sudoku(i): #returns an image of just the grid from an image containing a sudoku
    image = cv.imread(images[i])
    image = cv.resize(image,(0,0),fx=0.25,fy=0.25) #resize

    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY) #to grayscale
    blurred = cv.GaussianBlur(gray, (11,11), 3) #blur

    #threshold to get grid
    thresh = cv.adaptiveThreshold(blurred, 255,cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 11, 2)
    thresh = cv.bitwise_not(thresh)

    # find contours in the thresholded image
    cnts = cv.findContours(thresh.copy(), cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    #sort by biggest contour first
    cnts = sorted(cnts, key=cv.contourArea, reverse=True)
    
    sudokuCnt = None
    for c in cnts:
        # approximate the contour
        peri = cv.arcLength(c, True)
        approx = cv.approxPolyDP(c, 0.02 * peri, True)
        # check to see if it has 4 points
        if len(approx) == 4:
            sudokuCnt = approx
            break
    
    # apply a four point perspective transform to original image
    # automaticaly sorts points
    sudoku = four_point_transform(image, sudokuCnt.reshape(4, 2))
    return sudoku


In [4]:
def find_color(i):
    img = find_sudoku(i)

    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY) #gray
    gray = cv.GaussianBlur(gray, (3,3),2) #blur
#     print_img('s',gray)
    
    thresh = cv.adaptiveThreshold(gray, 255,cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 11, 2)
    thresh = cv.bitwise_not(thresh)
    
    #erode and dilate to extract the thick lines
    # needed extra accuracy for the aproach below
    
    erode_kernel = cv.getStructuringElement(cv.MORPH_RECT,(3,3))
    eroded = cv.erode(thresh, erode_kernel,iterations=2) 
#     print_img('e',eroded)   
    
    erode_kernel = cv.getStructuringElement(cv.MORPH_RECT,(3,3))
    dilated = cv.dilate(eroded, erode_kernel,iterations=5)
#     print_img('d',dilated)
    
    erode_kernel = cv.getStructuringElement(cv.MORPH_RECT,(4,4))
    eroded = cv.erode(dilated, erode_kernel,iterations=1) 
#     print_img('e',eroded)
    
    erode_kernel = cv.getStructuringElement(cv.MORPH_RECT,(3,3))
    dilated = cv.dilate(eroded, erode_kernel,iterations=4)
#     print_img('d',dilated)
    
    erode_kernel = cv.getStructuringElement(cv.MORPH_RECT,(6,6))
    eroded = cv.erode(dilated, erode_kernel,iterations=2) 
#     print_img('e',eroded)
    

    # Aproach:
    # flood fill each region with a different color
    
    h, w = eroded.shape[:2]
    dict_colors = {
        '1':(255,0,0),
        '2':(255,165,0),
        '3':(255,255,0),
        '4':(0,255,0),
        '5':(0,0,255),
        '6':(75,0,130),
        '7':(238,130,238),
        '8':(127,127,127),
        '9':(125,125,255)
    }
    
    # create a white border to close off the ends of regions which might have not reached the ends of the image
    color = cv.cvtColor(eroded, cv.COLOR_GRAY2BGR)
    color = cv.line(color,(0,0),(0,h),(255,255,255),20)
    color = cv.line(color,(0,0),(w,0),(255,255,255),20)
    color = cv.line(color,(w,0),(w,h),(255,255,255),20)
    color = cv.line(color,(0,h),(w,h),(255,255,255),20)
    
    #the sizes of cells
    hstep=int(h/9)
    wstep=int(w/9)
    
    # cover the middles of the cells with a black dot to get rid of any noise left 
    #in order to make sure that we will flood fill the black areas
    for j in range(9):
        for k in range(9):
            color = cv.circle(color, (int(hstep/2+hstep*j),int(wstep/2+wstep*k)), 16, (0,0,0), -1)

    # flood fill all black areas. go to next color after each step
    # if the color of the middle of the cell is black, flood fill the area with a color based on the counter
    counter = 0
    for j in range(9):
        for k in range(9):
            center = (int(wstep/2+wstep*j),int(hstep/2+hstep*k))
            x,y,z = color[center]
            if [x,y,z] == [0,0,0]:
                counter+=1
                mask = np.zeros((h+2, w+2), np.uint8)
                cv.floodFill(color, mask, (int(hstep/2+hstep*k),int(wstep/2+wstep*j)), dict_colors[str(counter)])

    return color

In [5]:
def check_digit(cell):
    # clear borders
    thresh = cv.threshold(cell, 0, 255,cv.THRESH_BINARY_INV | cv.THRESH_OTSU)[1]
    thresh = clear_border(thresh)
    
    # find contours in the thresholded cell
    cnts = cv.findContours(thresh.copy(), cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    # if no contours were found than this is an empty cell
    if len(cnts) == 0:
        return None
    
    
    # mask to check for digit
    c = max(cnts, key=cv.contourArea)
    mask = np.zeros(thresh.shape, dtype="uint8")
    cv.drawContours(mask, [c], -1, 255, -1)   
    
    # less than 2% of the mask is filled => the cell is empty
    (h, w) = thresh.shape
    percentFilled = cv.countNonZero(mask) / float(w * h)
    
    if percentFilled < 0.02:
        return None
    return 1

In [6]:

def answer(k,f):
    # use the flood filled color to get the number of the region
    dict_colors = {
        '[255   0   0]':1,
        '[255 165   0]':2,
        '[255 255   0]':3,
        '[  0 255   0]':4,
        '[  0   0 255]':5,
        '[ 75   0 130]':6,
        '[238 130 238]':7,
        '[127 127 127]':8,
        '[125 125 255]':9
    }

    #find the sudoku
    img = find_sudoku(k) #image with numbers
    #color all regions a different color
    color = find_color(k) # image with regions
    print_img('a',color)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    h,w=gray.shape
    #the sizes of a cell
    h_step = int(h/9)
    w_step = int(w/9)
    #go through grid and check color and if the cell is empty
    for i in range(9):
        for j in range(9):
            crop=gray[i*h_step:(i+1)*h_step,j*w_step:(j+1)*w_step]
            colorcrop=color[int((i*h_step+(i+1)*h_step)/2),int((j*w_step+(j+1)*w_step)/2)]
            if check_digit(crop) is None:
                f.write(str(dict_colors[str(colorcrop)])+'o')
            else:
                f.write(str(dict_colors[str(colorcrop)])+'x')
                
        if i<8:
            f.write('\n')
    f.close()

In [ ]:
base_folder = "test\\jigsaw\\" #change to "test\\jigsaw\\" for test images
output_folder = "evaluation\\submission_files\\Partu_Ana_Maria_407\\jigsaw\\"
images = glob.glob(os.path.join(base_folder,'*.jpg'))

for i in range(40):
    with open(output_folder+images[i][12:-4]+"_predicted.txt","w") as f:
        answer(i,f)